In [1]:
# Author: Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

In [2]:
import pandas as pd

# 数据集第一列是单词，第二列是词性，第三列是语法，第四列是实体标签。在NER任务中，只关心第一列和第四列。

df = pd.read_csv('data/train.tsv/eng.train',sep=' ')
df = df[df.columns[[0,3]]]

# 将单个单词根据.分隔成句子
df = df.dropna()
train_d  =df['-DOCSTART-'].values.tolist()
train_t = df['O.1'].values.tolist()

print(len(train_d),len(train_t))


train_data = []
train_tag = []

sent = []
tag = []
for i in range(len(train_d)):
    if train_d[i] != ".":
        if train_d != '':
            sent.append(train_d[i].lower())
            tag.append(train_t[i])
    else:
        if sent != []:
            train_data.append((sent,tag))
        sent = []
        tag = []


202385 202385


In [3]:
len(train_data)

7371

In [4]:
df = pd.read_csv('data/test.tsv/eng.testa',sep=' ')
df = df[df.columns[[0,3]]]

# 将单个单词根据.分隔成句子
df = df.dropna()
test_d  =df['-DOCSTART-'].values.tolist()
test_t = df['O.1'].values.tolist()

print(len(test_d),len(test_t))


test_data = []
test_tag = []

sent = []
tag = []
for i in range(len(test_d)):
    if test_d[i] != ".":
        if test_d != '':
            sent.append(test_d[i].lower())
            tag.append(test_t[i])
    else:
        if sent != []:
            test_data.append((sent,tag))
        sent = []
        tag = []

50934 50934


In [5]:
len(test_data)

1874

In [6]:
from importlib import import_module

model_name = 'BiLSTM_CRF'

x = import_module('models.' + model_name)
config = x.Config()
print('all class number : ',config.num_classes)



all class number :  9


In [7]:
from utils import build_dataset,MyDataset,DataLoader
vocab,train_da,train_la = build_dataset(config,train_data)
train_dataset = MyDataset(train_da,train_la)
train_loader = DataLoader(train_dataset,batch_size=config.batchsize,shuffle=True)

vocab,test_da,test_la = build_dataset(config,test_data)
test_dataset = MyDataset(test_da,test_la)
test_loader = DataLoader(test_dataset,batch_size=config.batchsize,shuffle=True)

config.num_vocab = len(vocab)

Loading exist vocab ... 
Vocab size: 10002
Loading exist vocab ... 
Vocab size: 10002


In [8]:
for data,label in train_dataset:
    print(data)
    print(label)
    break

tensor([  986, 10000,   202,   626,     4,  3936,   213,  5771])
tensor([2, 0, 4, 0, 0, 0, 4, 0])


In [9]:
from train import train

model = x.Model(config).to(config.device)

train(config,model,train_dataset,test_dataset,test_dataset)

Epoch [1/3]


100%|██████████| 1874/1874 [00:54<00:00, 34.25it/s]


train loss : 14.0031 ,train acc:0.894 ,dev acc : 0.889 
Epoch [2/3]


  9%|▉         | 700/7371 [01:03<10:01, 11.09it/s]


KeyboardInterrupt: 